In [87]:
%pip install kagglehub numpy pandas seaborn matplotlib scikit-learn graphviz


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [88]:
# features need to be class (convert numerical with binning/distribution)
# zero-frequency problem
# features are conditionally independent given the class

# I didn't even train test split some of the models and the accuracy is still bad??

In [89]:
from math import sqrt, pi, exp

# --- NaiveBayesClassifier From Scratch ---
class NaiveBayesClassifier:
    def __init__(self, categorical_features, numerical_features, target_feature, use_add1_smoothing=True):
        self.categorical_features = categorical_features
        self.numerical_features = numerical_features
        self.target_feature = target_feature
        self.use_add1_smoothing = use_add1_smoothing

        self.classes = []
        self.prior_probs = {} # P(Class)
        self.likelihoods_categorical = {} # P(Feature_cat | Class) with smoothing
        self.numerical_features_stats = {} # {Class: {Feature_num: {'mean': .., 'std': ..}}}
        self.feature_possible_values = {} # For smoothing denominator

    def fit(self, df):
        """
        ฝึกโมเดล Naive Bayes โดยคำนวณ Prior, Likelihoods, และ Stats สำหรับ Numerical Features
        **Likelihoods สำหรับ Categorical Predictors จะไม่ใช้ Add-1 Smoothing**
        """
        self.classes = df[self.target_feature].unique()
        total_samples = len(df)

        # 1. คำนวณ Prior Probabilities
        for cls in self.classes:
            self.prior_probs[cls] = df[df[self.target_feature] == cls].shape[0] / total_samples

        # 2. คำนวณ Likelihoods สำหรับ Categorical Predictors (ไม่มี Smoothing)
        for feature in self.categorical_features:
            self.feature_possible_values[feature] = df[feature].nunique() # ไม่ได้ใช้แล้วถ้าไม่ smoothing
            for cls in self.classes:
                if cls not in self.likelihoods_categorical:
                    self.likelihoods_categorical[cls] = {}
                self.likelihoods_categorical[cls][feature] = {}

                df_class = df[df[self.target_feature] == cls]
                total_count_class = df_class.shape[0]

                for feat_value in df[feature].unique():
                    count_feat_class = df_class[feature].value_counts().get(feat_value, 0)

                    # *** จุดที่แก้ไข 2: ใช้เงื่อนไข use_add1_smoothing ***
                    if self.use_add1_smoothing:
                        prob = (count_feat_class + 1) / (total_count_class + self.feature_possible_values[feature])
                    else:
                        if total_count_class > 0:
                            prob = count_feat_class / total_count_class
                        else:
                            prob = 0.0 # ถ้าไม่มีข้อมูลในคลาสนี้เลย ให้ Likelihood เป็น 0

                    self.likelihoods_categorical[cls][feature][feat_value] = prob

        # 3. คำนวณ Mean และ Standard Deviation สำหรับ Numerical Predictors
        for feature in self.numerical_features:
            for cls in self.classes:
                if cls not in self.numerical_features_stats:
                    self.numerical_features_stats[cls] = {}

                df_class_feature = df[df[self.target_feature] == cls][feature]
                self.numerical_features_stats[cls][feature] = {
                    'mean': df_class_feature.mean(),
                    'std': df_class_feature.std(ddof=1) # ddof=1 for sample std dev
                }
                # Handle cases where std dev is 0 (e.g., all values are same in a class)
                if self.numerical_features_stats[cls][feature]['std'] == 0:
                    self.numerical_features_stats[cls][feature]['std'] = 1e-9 # Prevent division by zero, small epsilon

    def _normal_pdf(self, x, mean, std):
        """Probability Density Function for Normal Distribution."""
        if std == 0: # Should be handled by epsilon in fit, but as a safeguard
            return 1.0 if x == mean else 0.0
        exponent = exp(-((x - mean) ** 2) / (2 * (std ** 2)))
        return (1 / (sqrt(2 * pi) * std)) * exponent

    def _calculate_unnormalized_posterior_with_terms(self, query_features, target_label):
        """
        คำนวณ Numerator (Score) สำหรับคลาสที่ระบุ พร้อมส่งกลับเทอมการคูณแต่ละตัว
        Score = P(X|c) * P(c)
        """
        score = self.prior_probs[target_label]
        terms = [f"{self.prior_probs[target_label]:.4f}"] # เก็บเฉพาะตัวเลขสำหรับส่วน (0.XX * 0.YY...)

        # สำหรับแสดง P(Yes) = ... หรือ P(No) = ... ในบรรทัดแรกของ P(X|c)*P(c)
        detailed_terms_display = [f"P({target_label}) = {self.prior_probs[target_label]:.4f}"]

        for feat_name, feat_value in query_features.items():
            if feat_name in self.categorical_features:
                p_feat_given_label = self.likelihoods_categorical[target_label][feat_name].get(feat_value, 0)
                score *= p_feat_given_label

                display_value = feat_value
                terms.append(f"{p_feat_given_label:.4f}")
                detailed_terms_display.append(f"P({feat_name}={display_value}|{target_label}) = {p_feat_given_label:.4f}")
            elif feat_name in self.numerical_features:
                mean = self.numerical_features_stats[target_label][feat_name]['mean']
                std = self.numerical_features_stats[target_label][feat_name]['std']
                p_feat_given_label = self._normal_pdf(feat_value, mean, std)
                score *= p_feat_given_label
                terms.append(f"{p_feat_given_label:.4f}")
                detailed_terms_display.append(f"PDF({feat_name}={feat_value}|{target_label}) = {p_feat_given_label:.4f}")
            else:
                # This should ideally not happen if features are correctly defined
                print(f"Warning: Feature '{feat_name}' not recognized during prediction. Skipping.")
        return score, terms, detailed_terms_display

    def predict_proba(self, query_features):
        """
        ทำนาย Posterior Probabilities สำหรับแต่ละคลาส
        คืนค่าเป็น dictionary {class: probability}
        """
        scores = {}
        # We don't need detailed terms for predict_proba, only for display
        for cls in self.classes:
            score, _, _ = self._calculate_unnormalized_posterior_with_terms(query_features, cls)
            scores[cls] = score

        total_score = sum(scores.values())

        posterior_probs = {}
        if total_score > 0:
            for cls in self.classes:
                posterior_probs[cls] = scores[cls] / total_score
        else:
            for cls in self.classes:
                posterior_probs[cls] = 0.0
            print("Warning: All unnormalized scores are zero. Posterior probabilities set to 0.0.")

        return posterior_probs

    def predict(self, query_features):
        """
        ทำนายคลาสที่มีความน่าจะเป็นสูงสุด
        """
        posterior_probs = self.predict_proba(query_features)

        if posterior_probs:
            return max(posterior_probs, key=posterior_probs.get)
        else:
            return None

In [90]:
import pandas as pd

dataset_path = "../Cleaned_Breast_Cancer.csv"
df = pd.read_csv(dataset_path)

df.head()

,Age,Grade,Tumor Size,Survival Months,Node Positive Ratio,T Stage Encoded,M Stage Encoded,N Stage Encoded,Estrogen Status Encoded,Progesterone Status Encoded,Survival More Than 6 Years
0,68,3,4,60,0.041667,0.0,1,0.0,1,1,0
1,50,2,35,62,0.357143,1.0,1,1.0,1,1,0
2,58,2,63,75,0.500000,2.0,1,2.0,1,1,1
3,58,3,18,84,0.500000,0.0,1,0.0,1,1,1
4,47,3,41,50,0.333333,1.0,1,0.0,1,1,0


In [91]:
# Is this necessary?
# Make categorical features categorical again

In [92]:
# 1. กำหนด Features และ Target
categorical_feats = [
    "Grade",
    "T Stage Encoded",
    "M Stage Encoded",
    "N Stage Encoded",
    "Estrogen Status Encoded",
    "Progesterone Status Encoded",
]
numerical_feats = [
    "Age",
    "Tumor Size",
    "Node Positive Ratio",
]
target_feat = "Survival More Than 6 Years"

X = df.drop(columns=["Survival Months", "Survival More Than 6 Years"])
y = df["Survival More Than 6 Years"]

# 2. fit model
nb_model = NaiveBayesClassifier(categorical_feats, numerical_feats, target_feat, use_add1_smoothing=False)
nb_model.fit(df)

In [93]:
# Step 1 : Calculate Prior Probability of Classes P(C)
print(f"\nP(1) = {nb_model.prior_probs[1]:.4f}")
print(f"P(0) = {nb_model.prior_probs[0]:.4f}")


P(1) = 0.5052
P(0) = 0.4948


In [94]:
# Step 2 : Calculate the Likelihood Table for all features

total_yes_samples = df[df[target_feat] == 1].shape[0]
total_no_samples = df[df[target_feat] == 0].shape[0]

for feature_name in categorical_feats:
    print(f"\n=== Likelihood Table for {feature_name.capitalize()} ===")
    num_unique_feature_values = df[feature_name].nunique()
    for feat_value in sorted(df[feature_name].unique()):
        display_feat_value = feat_value

        count_feat_class_yes = df[
            (df[target_feat] == 1) & (df[feature_name] == feat_value)
        ].shape[0]
        count_feat_class_no = df[
            (df[target_feat] == 0) & (df[feature_name] == feat_value)
        ].shape[0]

        denominator_yes = total_yes_samples + num_unique_feature_values
        denominator_no = total_no_samples + num_unique_feature_values

        if nb_model.use_add1_smoothing:
            fraction_yes_str = f"{count_feat_class_yes + 1}/{total_yes_samples + num_unique_feature_values}"
            fraction_no_str = f"{count_feat_class_no + 1}/{total_no_samples + num_unique_feature_values}"
        else:  # กรณีไม่มี smoothing
            fraction_yes_str = (
                f"{count_feat_class_yes}/{total_yes_samples}"
                if total_yes_samples > 0
                else "0/0"
            )
            fraction_no_str = (
                f"{count_feat_class_no}/{total_no_samples}"
                if total_no_samples > 0
                else "0/0"
            )

        yes_prob = nb_model.likelihoods_categorical[1][feature_name].get(
            feat_value, 0
        )
        no_prob = nb_model.likelihoods_categorical[0][feature_name].get(
            feat_value, 0
        )

        print(
            f"{str(display_feat_value):9s} | P({str(display_feat_value):9s} | Yes) : {fraction_yes_str:<10s} = {yes_prob:.2f} | P({str(display_feat_value):9s} | No) : {fraction_no_str:<10s} = {no_prob:.2f}"
        )


=== Likelihood Table for Grade ===
1         | P(1         | Yes) : 285/2033   = 0.14 | P(1         | No) : 258/1991   = 0.13
2         | P(2         | Yes) : 1206/2033  = 0.59 | P(2         | No) : 1145/1991  = 0.58
3         | P(3         | Yes) : 532/2033   = 0.26 | P(3         | No) : 579/1991   = 0.29
4         | P(4         | Yes) : 10/2033    = 0.00 | P(4         | No) : 9/1991     = 0.00

=== Likelihood Table for T stage encoded ===
0.0       | P(0.0       | Yes) : 855/2033   = 0.42 | P(0.0       | No) : 748/1991   = 0.38
1.0       | P(1.0       | Yes) : 881/2033   = 0.43 | P(1.0       | No) : 905/1991   = 0.45
2.0       | P(2.0       | Yes) : 245/2033   = 0.12 | P(2.0       | No) : 288/1991   = 0.14
3.0       | P(3.0       | Yes) : 52/2033    = 0.03 | P(3.0       | No) : 50/1991    = 0.03

=== Likelihood Table for M stage encoded ===
0         | P(0         | Yes) : 35/2033    = 0.02 | P(0         | No) : 57/1991    = 0.03
1         | P(1         | Yes) : 1998/2033  = 0.98 | 

In [95]:
# 3. Set query
# Pandas DataFrame X into a list of dictionaries (objects)
querys = X.to_dict(orient="records")
querys

[{'Age': 68,
  'Grade': 3,
  'Tumor Size': 4,
  'Node Positive Ratio': 0.0416666666666666,
  'T Stage Encoded': 0.0,
  'M Stage Encoded': 1,
  'N Stage Encoded': 0.0,
  'Estrogen Status Encoded': 1,
  'Progesterone Status Encoded': 1},
 {'Age': 50,
  'Grade': 2,
  'Tumor Size': 35,
  'Node Positive Ratio': 0.3571428571428571,
  'T Stage Encoded': 1.0,
  'M Stage Encoded': 1,
  'N Stage Encoded': 1.0,
  'Estrogen Status Encoded': 1,
  'Progesterone Status Encoded': 1},
 {'Age': 58,
  'Grade': 2,
  'Tumor Size': 63,
  'Node Positive Ratio': 0.5,
  'T Stage Encoded': 2.0,
  'M Stage Encoded': 1,
  'N Stage Encoded': 2.0,
  'Estrogen Status Encoded': 1,
  'Progesterone Status Encoded': 1},
 {'Age': 58,
  'Grade': 3,
  'Tumor Size': 18,
  'Node Positive Ratio': 0.5,
  'T Stage Encoded': 0.0,
  'M Stage Encoded': 1,
  'N Stage Encoded': 0.0,
  'Estrogen Status Encoded': 1,
  'Progesterone Status Encoded': 1},
 {'Age': 47,
  'Grade': 3,
  'Tumor Size': 41,
  'Node Positive Ratio': 0.333333333

In [96]:
# 4. Posterior Probabilities (Unnormalized)
probabilities = []

for query in querys:
    score_yes, terms_numerical_yes, terms_display_yes = (
        nb_model._calculate_unnormalized_posterior_with_terms(query, 1)
    )

    # Display with 4 decimal places for consistency
    formatted_numerical_terms_yes = [
        f"{float(term):.4f}" for term in terms_numerical_yes
    ]

    score_no, terms_numerical_no, terms_display_no = (
        nb_model._calculate_unnormalized_posterior_with_terms(query, 0)
    )

    # Display with 4 decimal places for consistency
    formatted_numerical_terms_no = [f"{float(term):.4f}" for term in terms_numerical_no]

    probabilities.append((score_yes, score_no))

In [104]:
# 5. Final Decision (Normalized Posterior Probabilities)
import numpy as np

prob_trues = []

for prob in probabilities:
    (score_yes, score_no) = prob
    total_unnormalized_score = score_yes + score_no

    posterior_yes_normalized = (
        (score_yes / total_unnormalized_score) * 100
        if total_unnormalized_score > 0
        else 0
    )
    posterior_no_normalized = (
        (score_no / total_unnormalized_score) * 100
        if total_unnormalized_score > 0
        else 0
    )

    prob_trues.append(posterior_yes_normalized)

    # print("\n--- Final Decision ---")
    # print(f"P(Play=Yes | x) = {score_yes:.4f} ({posterior_yes_normalized:.2f}%)")
    # print(f"P(Play=No  | x) = {score_no:.4f} ({posterior_no_normalized:.2f}%)")

prob_trues = np.array(prob_trues)
y_pred = (prob_trues >= 50).astype(int)
y_pred
# prob_trues

array([1, 1, 0, ..., 1, 1, 1], shape=(4024,))

In [107]:
# train test split
# accuracy score from sklearn

from sklearn.metrics import accuracy_score

score = accuracy_score(y, y_pred)
score

0.5320576540755467